# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import yfinance as yf
import pandas as pd
import math
from scipy.stats import percentileofscore as score
from statistics import mean
from IPython.display import clear_output

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis. But, here we have just taken a list of ~500 stocks in a CSV file taken at the time of building the project.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp500.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
498,YUM
499,ZBH
500,ZBRA
501,ZION


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame.

In [3]:
my_columns = ['Ticker', 'Stock Price', '1 Year Return', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy


## Looping Through The Tickers in Our List of Stocks

Here to fetch the stock data, we have used `yfinance` API

We can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [4]:
for stock in stocks['Ticker']:
    data = yf.Ticker(stock).info
    try:
        yr1Return = data['52WeekChange']
    except:
        yr1Return = np.NaN
    stock_dict = {
        'Ticker': [stock],
        'Stock Price': [data['currentPrice']],
        '1 Year Return': [yr1Return],
        'Number of Shares to Buy': ['N/A']
    }
    df = pd.DataFrame(stock_dict)
    final_dataframe = pd.concat([final_dataframe, df], ignore_index=True)

    clear_output(wait=True)
    print(final_dataframe)

    Ticker  Stock Price  1 Year Return Number of Shares to Buy
0        A       126.30      -0.046936                     N/A
1      AAL        15.84       0.054594                     N/A
2      AAP        71.01      -0.634515                     N/A
3     AAPL       181.99       0.103839                     N/A
4     ABBV       147.73       0.052658                     N/A
..     ...          ...            ...                     ...
498    YUM       134.35            NaN                     N/A
499    ZBH       126.32            NaN                     N/A
500   ZBRA       251.41            NaN                     N/A
501   ZION        37.92            NaN                     N/A
502    ZTS       178.63            NaN                     N/A

[503 rows x 4 columns]


In [5]:
final_dataframe

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy
0,A,126.30,-0.046936,N/A
1,AAL,15.84,0.054594,N/A
2,AAP,71.01,-0.634515,N/A
3,AAPL,181.99,0.103839,N/A
4,ABBV,147.73,0.052658,N/A
...,...,...,...,...
498,YUM,134.35,NaN,N/A
499,ZBH,126.32,NaN,N/A
500,ZBRA,251.41,NaN,N/A
501,ZION,37.92,NaN,N/A


In [6]:
final_dataframe[final_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy
80,CAT,276.44,NaN,N/A
353,ORCL,114.44,NaN,N/A
354,ORLY,925.93,NaN,N/A
355,OTIS,88.51,NaN,N/A
356,OXY,63.52,NaN,N/A
...,...,...,...,...
498,YUM,134.35,NaN,N/A
499,ZBH,126.32,NaN,N/A
500,ZBRA,251.41,NaN,N/A
501,ZION,37.92,NaN,N/A


Filling the `NULL` cells of the `1 Year Return` column with mean value

In [7]:
final_dataframe['1 Year Return'].fillna(final_dataframe['1 Year Return'].mean(), inplace=True)

In [8]:
final_dataframe[final_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy


In [9]:
final_dataframe

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy
0,A,126.30,-0.046936,N/A
1,AAL,15.84,0.054594,N/A
2,AAP,71.01,-0.634515,N/A
3,AAPL,181.99,0.103839,N/A
4,ABBV,147.73,0.052658,N/A
...,...,...,...,...
498,YUM,134.35,0.050348,N/A
499,ZBH,126.32,0.050348,N/A
500,ZBRA,251.41,0.050348,N/A
501,ZION,37.92,0.050348,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.

In [10]:
final_dataframe.sort_values('1 Year Return', ascending=False, inplace=True)

In [11]:
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop=True, inplace=True)

In [12]:
final_dataframe

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy
0,NVDA,446.80,1.511100,N/A
1,GE,113.06,0.926441,N/A
2,NFLX,431.60,0.848473,N/A
3,META,310.73,0.825140,N/A
4,FSLR,194.38,0.821061,N/A
5,ACGL,76.72,0.737712,N/A
6,FICO,841.02,0.716928,N/A
7,CCL,17.16,0.714286,N/A
8,DHI,125.71,0.627103,N/A
9,AVGO,881.65,0.616135,N/A


## Calculating the Number of Shares to Buy

As we can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

In [13]:
# portfolio_size = float(input("Enter your portfolio value: "))
portfolio_size = 10000000
portfolio_size

10000000

In [14]:
position_size = float(portfolio_size) / len(final_dataframe.index)
position_size

200000.0

In [15]:
for i in range(len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(
        position_size / final_dataframe.loc[i, 'Stock Price'])

final_dataframe

,Ticker,Stock Price,1 Year Return,Number of Shares to Buy
0,NVDA,446.80,1.511100,447
1,GE,113.06,0.926441,1768
2,NFLX,431.60,0.848473,463
3,META,310.73,0.825140,643
4,FSLR,194.38,0.821061,1028
5,ACGL,76.72,0.737712,2606
6,FICO,841.02,0.716928,237
7,CCL,17.16,0.714286,11655
8,DHI,125.71,0.627103,1590
9,AVGO,881.65,0.616135,226


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is that low-quality momentum can often be caused by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Here `hqm` stands for `high-quality momentum`.

In [16]:
hqm_columns = [
    'Ticker',
    'Stock Price',
    'Number of Shares to Buy',
    '1 Year Return',
    '1 Year Return Percentile',
    '6 Months Return',
    '6 Months Return Percentile',
    '3 Months Return',
    '3 Months Return Percentile',
    '1 Month Return',
    '1 Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns=hqm_columns)
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score


In [17]:
for stock in stocks['Ticker']:
    data = yf.Ticker(stock).info
    try:
        yr1Return = data['52WeekChange']
    except:
        yr1Return = np.NaN
    try:
        hist = yf.Ticker(stock).history(period='6mo')
        mnth6Return = ((hist['Close'][-1] - hist['Close'][0]) / hist['Close'][0]) * 100
    except:
        mnth6Return = np.NaN
    try:
        hist = yf.Ticker(stock).history(period='3mo')
        mnth3Return = ((hist['Close'][-1] - hist['Close'][0]) / hist['Close'][0]) * 100
    except:
        mnth3Return = np.NaN
    try:
        hist = yf.Ticker(stock).history(period='1mo')
        mnth1Return = ((hist['Close'][-1] - hist['Close'][0]) / hist['Close'][0]) * 100
    except:
        mnth1Return = np.NaN
    
    stock_dict = {
        'Ticker': [stock],
        'Stock Price': [data['currentPrice']],
        'Number of Shares to Buy': ['N/A'],
        '1 Year Return': [yr1Return],
        '1 Year Return Percentile': ['N/A'],
        '6 Months Return': [mnth6Return],
        '6 Months Return Percentile': ['N/A'],
        '3 Months Return': [mnth3Return],
        '3 Months Return Percentile': ['N/A'],
        '1 Month Return': [mnth1Return],
        '1 Month Return Percentile': ['N/A'],
        'HQM Score': ['N/A']
    }
    df = pd.DataFrame(stock_dict)
    hqm_dataframe = pd.concat([hqm_dataframe, df], ignore_index=True)

    clear_output(wait=True)
    print(hqm_dataframe['Ticker'])

0         A
1       AAL
2       AAP
3      AAPL
4      ABBV
       ... 
498     YUM
499     ZBH
500    ZBRA
501    ZION
502     ZTS
Name: Ticker, Length: 503, dtype: object


In [18]:
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,A,126.30,N/A,NaN,N/A,-17.038887,N/A,-4.643257,N/A,6.645278,N/A,N/A
1,AAL,15.84,N/A,NaN,N/A,-6.382978,N/A,10.306410,N/A,-11.409391,N/A,N/A
2,AAP,71.01,N/A,NaN,N/A,-53.181247,N/A,-43.530813,N/A,1.500862,N/A,N/A
3,AAPL,181.99,N/A,NaN,N/A,19.943327,N/A,4.893375,N/A,-5.119646,N/A,N/A
4,ABBV,147.73,N/A,NaN,N/A,1.868702,N/A,0.189894,N/A,7.635698,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,134.35,N/A,NaN,N/A,4.098875,N/A,-1.596713,N/A,0.336081,N/A,N/A
499,ZBH,126.32,N/A,NaN,N/A,-1.711799,N/A,-8.090805,N/A,-10.538242,N/A,N/A
500,ZBRA,251.41,N/A,NaN,N/A,-22.571606,N/A,-9.519176,N/A,-13.646359,N/A,N/A
501,ZION,37.92,N/A,NaN,N/A,-29.907580,N/A,56.306669,N/A,36.501070,N/A,N/A


In [19]:
hqm_dataframe[hqm_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,A,126.30,N/A,NaN,N/A,-17.038887,N/A,-4.643257,N/A,6.645278,N/A,N/A
1,AAL,15.84,N/A,NaN,N/A,-6.382978,N/A,10.306410,N/A,-11.409391,N/A,N/A
2,AAP,71.01,N/A,NaN,N/A,-53.181247,N/A,-43.530813,N/A,1.500862,N/A,N/A
3,AAPL,181.99,N/A,NaN,N/A,19.943327,N/A,4.893375,N/A,-5.119646,N/A,N/A
4,ABBV,147.73,N/A,NaN,N/A,1.868702,N/A,0.189894,N/A,7.635698,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,134.35,N/A,NaN,N/A,4.098875,N/A,-1.596713,N/A,0.336081,N/A,N/A
499,ZBH,126.32,N/A,NaN,N/A,-1.711799,N/A,-8.090805,N/A,-10.538242,N/A,N/A
500,ZBRA,251.41,N/A,NaN,N/A,-22.571606,N/A,-9.519176,N/A,-13.646359,N/A,N/A
501,ZION,37.92,N/A,NaN,N/A,-29.907580,N/A,56.306669,N/A,36.501070,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data which we have to fill with `mean` values.

In [20]:
hqm_dataframe['1 Year Return'].fillna(hqm_dataframe['1 Year Return'].mean(), inplace=True)

In [21]:
hqm_dataframe[hqm_dataframe.isnull().any(axis=1)]

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,A,126.30,N/A,NaN,N/A,-17.038887,N/A,-4.643257,N/A,6.645278,N/A,N/A
1,AAL,15.84,N/A,NaN,N/A,-6.382978,N/A,10.306410,N/A,-11.409391,N/A,N/A
2,AAP,71.01,N/A,NaN,N/A,-53.181247,N/A,-43.530813,N/A,1.500862,N/A,N/A
3,AAPL,181.99,N/A,NaN,N/A,19.943327,N/A,4.893375,N/A,-5.119646,N/A,N/A
4,ABBV,147.73,N/A,NaN,N/A,1.868702,N/A,0.189894,N/A,7.635698,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,134.35,N/A,NaN,N/A,4.098875,N/A,-1.596713,N/A,0.336081,N/A,N/A
499,ZBH,126.32,N/A,NaN,N/A,-1.711799,N/A,-8.090805,N/A,-10.538242,N/A,N/A
500,ZBRA,251.41,N/A,NaN,N/A,-22.571606,N/A,-9.519176,N/A,-13.646359,N/A,N/A
501,ZION,37.92,N/A,NaN,N/A,-29.907580,N/A,56.306669,N/A,36.501070,N/A,N/A


In [22]:
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,A,126.30,N/A,NaN,N/A,-17.038887,N/A,-4.643257,N/A,6.645278,N/A,N/A
1,AAL,15.84,N/A,NaN,N/A,-6.382978,N/A,10.306410,N/A,-11.409391,N/A,N/A
2,AAP,71.01,N/A,NaN,N/A,-53.181247,N/A,-43.530813,N/A,1.500862,N/A,N/A
3,AAPL,181.99,N/A,NaN,N/A,19.943327,N/A,4.893375,N/A,-5.119646,N/A,N/A
4,ABBV,147.73,N/A,NaN,N/A,1.868702,N/A,0.189894,N/A,7.635698,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,134.35,N/A,NaN,N/A,4.098875,N/A,-1.596713,N/A,0.336081,N/A,N/A
499,ZBH,126.32,N/A,NaN,N/A,-1.711799,N/A,-8.090805,N/A,-10.538242,N/A,N/A
500,ZBRA,251.41,N/A,NaN,N/A,-22.571606,N/A,-9.519176,N/A,-13.646359,N/A,N/A
501,ZION,37.92,N/A,NaN,N/A,-29.907580,N/A,56.306669,N/A,36.501070,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

In [23]:
time_periods = [
    '1 Year',
    '6 Months',
    '3 Months',
    '1 Month'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = score(hqm_dataframe[change_col],
                                                                           hqm_dataframe.loc[row, change_col]) / 100

In [24]:
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,A,126.30,N/A,NaN,NaN,-17.038887,0.115308,-4.643257,0.212724,6.645278,0.751491,N/A
1,AAL,15.84,N/A,NaN,NaN,-6.382978,0.318091,10.306410,0.638171,-11.409391,0.049702,N/A
2,AAP,71.01,N/A,NaN,NaN,-53.181247,0.001988,-43.530813,0.001988,1.500862,0.481113,N/A
3,AAPL,181.99,N/A,NaN,NaN,19.943327,0.892644,4.893375,0.473161,-5.119646,0.161034,N/A
4,ABBV,147.73,N/A,NaN,NaN,1.868702,0.552684,0.189894,0.345924,7.635698,0.7833,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,134.35,N/A,NaN,NaN,4.098875,0.610338,-1.596713,0.290258,0.336081,0.425447,N/A
499,ZBH,126.32,N/A,NaN,NaN,-1.711799,0.467197,-8.090805,0.129225,-10.538242,0.057654,N/A
500,ZBRA,251.41,N/A,NaN,NaN,-22.571606,0.069583,-9.519176,0.099404,-13.646359,0.021869,N/A
501,ZION,37.92,N/A,NaN,NaN,-29.907580,0.027833,56.306669,0.998012,36.501070,1.0,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [25]:
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

In [26]:
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,A,126.30,N/A,NaN,NaN,-17.038887,0.115308,-4.643257,0.212724,6.645278,0.751491,NaN
1,AAL,15.84,N/A,NaN,NaN,-6.382978,0.318091,10.306410,0.638171,-11.409391,0.049702,NaN
2,AAP,71.01,N/A,NaN,NaN,-53.181247,0.001988,-43.530813,0.001988,1.500862,0.481113,NaN
3,AAPL,181.99,N/A,NaN,NaN,19.943327,0.892644,4.893375,0.473161,-5.119646,0.161034,NaN
4,ABBV,147.73,N/A,NaN,NaN,1.868702,0.552684,0.189894,0.345924,7.635698,0.7833,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,134.35,N/A,NaN,NaN,4.098875,0.610338,-1.596713,0.290258,0.336081,0.425447,NaN
499,ZBH,126.32,N/A,NaN,NaN,-1.711799,0.467197,-8.090805,0.129225,-10.538242,0.057654,NaN
500,ZBRA,251.41,N/A,NaN,NaN,-22.571606,0.069583,-9.519176,0.099404,-13.646359,0.021869,NaN
501,ZION,37.92,N/A,NaN,NaN,-29.907580,0.027833,56.306669,0.998012,36.501070,1.0,NaN


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [27]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)

In [28]:
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop=True, inplace=True)

In [29]:
hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,A,126.30,N/A,NaN,NaN,-17.038887,0.115308,-4.643257,0.212724,6.645278,0.751491,NaN
1,AAL,15.84,N/A,NaN,NaN,-6.382978,0.318091,10.306410,0.638171,-11.409391,0.049702,NaN
2,AAP,71.01,N/A,NaN,NaN,-53.181247,0.001988,-43.530813,0.001988,1.500862,0.481113,NaN
3,AAPL,181.99,N/A,NaN,NaN,19.943327,0.892644,4.893375,0.473161,-5.119646,0.161034,NaN
4,ABBV,147.73,N/A,NaN,NaN,1.868702,0.552684,0.189894,0.345924,7.635698,0.7833,NaN
5,ABC,185.33,N/A,NaN,NaN,19.962454,0.89662,10.177751,0.634195,-3.569385,0.222664,NaN
6,ABT,107.65,N/A,NaN,NaN,-2.799096,0.435388,-2.930568,0.266402,0.419780,0.435388,NaN
7,ACGL,76.72,N/A,NaN,NaN,23.423426,0.918489,0.761757,0.361829,3.858127,0.604374,NaN
8,ACN,312.00,N/A,NaN,NaN,9.673787,0.761431,17.815872,0.831014,1.308569,0.471173,NaN
9,ADBE,526.88,N/A,NaN,NaN,40.415209,0.988072,53.136083,0.994036,9.472459,0.840954,NaN


## Calculating the Number of Shares to Buy

As we can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

In [30]:
# portfolio_size = float(input("Enter your portfolio value: "))
portfolio_size = 10000000
portfolio_size

10000000

In [31]:
position_size = portfolio_size / len(final_dataframe.index)
position_size

200000.0

In [32]:
for i in range(len(hqm_dataframe.index)):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe.loc[i, 'Stock Price'])

hqm_dataframe

,Ticker,Stock Price,Number of Shares to Buy,1 Year Return,1 Year Return Percentile,6 Months Return,6 Months Return Percentile,3 Months Return,3 Months Return Percentile,1 Month Return,1 Month Return Percentile,HQM Score
0,A,126.30,1583,NaN,NaN,-17.038887,0.115308,-4.643257,0.212724,6.645278,0.751491,NaN
1,AAL,15.84,12626,NaN,NaN,-6.382978,0.318091,10.306410,0.638171,-11.409391,0.049702,NaN
2,AAP,71.01,2816,NaN,NaN,-53.181247,0.001988,-43.530813,0.001988,1.500862,0.481113,NaN
3,AAPL,181.99,1098,NaN,NaN,19.943327,0.892644,4.893375,0.473161,-5.119646,0.161034,NaN
4,ABBV,147.73,1353,NaN,NaN,1.868702,0.552684,0.189894,0.345924,7.635698,0.7833,NaN
5,ABC,185.33,1079,NaN,NaN,19.962454,0.89662,10.177751,0.634195,-3.569385,0.222664,NaN
6,ABT,107.65,1857,NaN,NaN,-2.799096,0.435388,-2.930568,0.266402,0.419780,0.435388,NaN
7,ACGL,76.72,2606,NaN,NaN,23.423426,0.918489,0.761757,0.361829,3.858127,0.604374,NaN
8,ACN,312.00,641,NaN,NaN,9.673787,0.761431,17.815872,0.831014,1.308569,0.471173,NaN
9,ADBE,526.88,379,NaN,NaN,40.415209,0.988072,53.136083,0.994036,9.472459,0.840954,NaN


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

### Initializing our XlsxWriter Object

In [33]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index=False)

## Formatting Our Excel Output

In [34]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_template = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_template = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

percent_template = writer.book.add_format(
    {
        'num_format': '0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [35]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', integer_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

In [36]:
writer.close()